In [1]:
pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 48.0 MB/s eta 0:00:00


In [2]:
pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
pip install qrcode

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tabula 
import pandas as pd 
from openpyxl import Workbook, load_workbook   
from openpyxl.drawing.image import Image
from tkinter import filedialog 
from datetime import date 
import qrcode


data_atual = date.today()

caminho_arquivo = filedialog.askopenfilenames()    

lista_tabelas = tabula.read_pdf(caminho_arquivo[0], area=[164.135,1.117,601.827,594.383], pages="all", stream=True, guess=True) 

lista_romaneio = lista_tabelas

x = len(lista_tabelas)
y = len(lista_romaneio)

print(y)   

if x == 0: 
    x = 1
    lista_tabelas = tabula.read_pdf(caminho_arquivo[0], area=[164.135,1.117,601.827,594.383], pages="all", stream=True, guess=True) 
    
    lista_romaneio = lista_tabelas
    
    display(lista_tabelas) 


for i in range(0, x): 
    lista_romaneio[i] = lista_tabelas[i]
    
    
for i in range(0, x):  
    
    
    y = len(lista_tabelas[i].index)  
    
    if pd.isnull(lista_tabelas[i].loc[0, "Item"]) == True: 
    
        lista_tabelas[i]["Item"] = lista_tabelas[i]["Unnamed: 0"] 
    
        if pd.isnull(lista_tabelas[i].loc[y - 1, "Item"]) == True: 
        
            display(lista_tabelas[i].loc[0:y-2, ["Item", "Qtde"]])
            lista_tabelas[i] = lista_tabelas[i].loc[0:y-2, ["Item", "Qtde"]] 
            lista_romaneio[i] = lista_tabelas[i] 
        
        else:
            display(lista_tabelas[i].loc[0: , ["Item", "Qtde"]]) 
            lista_tabelas[i] = lista_tabelas[i].loc[0: , ["Item", "Qtde"]]
            lista_romaneio[i] = lista_tabelas[i] 
    else: 
        if pd.isnull(lista_tabelas[i].loc[y - 1, "Item"]) == True: 
            display(lista_tabelas[i].loc[0:y-2, ["Item", "Qtde"]])
            lista_tabelas[i] = lista_tabelas[i].loc[0:y-2, ["Item", "Qtde"]]     
            lista_romaneio[i] = lista_tabelas[i]  
        
        else: 
            lista_romaneio[i] = lista_tabelas[i]

tabela_final = pd.concat(lista_romaneio, ignore_index=True)  

# Abre a planilha existente
planilha = load_workbook("PlanilhaRomaneio.xlsx") 

# Abre uma aba ativa do excel
aba_ativa = planilha.active 

#Pega o numero de linhas da tabela
x = len(tabela_final.index)


#Preenche na célula quantidade 
inc = 0
for i in range(8, 8 + x): 
    realocar = str(tabela_final.loc[inc, "Qtde"]).replace(',','.')
    aba_ativa[f"B{i}"] = float(realocar)
    inc += 1

# Preenche o Item 
inc = 0

for i in range(8, 8 + x): 
    aba_ativa[f"A{i}"] = tabela_final.loc[inc, "Item"]
    inc += 1 

aba_ativa["C7"] = data_atual.strftime("%d/%m/%Y")   
aba_ativa["B4"] = "DATA: " + data_atual.strftime("%d/%m/%Y")  

#Implatação QR Code

qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=7,
    border=1
)  

texto = ""

for i in range(0, x): 
    x1 = tabela_final.loc[i, "Item"] + " " + "\n" + str(tabela_final.loc[i, "Qtde"])+ " UND" + "\n\n" 
    texto += x1  

print(texto)
    
qr.add_data(texto)
qr.make(fit=True)

imagem = qr.make_image(fill_color='black', back_color='white')

imagem.save('qrcodeteste.png')


1


,Item,Qtde
0,TRELIÇA H8 - 7 MT,"100,000"
1,TRELIÇA H8 - 9 MT,"450,000"


TRELIÇA H8 - 7 MT 
100,000 UND

TRELIÇA H8 - 9 MT 
450,000 UND




In [5]:
planilha.save("Romaneio.xlsx")
